- In this notebook, we will get a high level overview of major topics in LLM field.
- In future notebooks, we will deepdive into each topics.

- Text we pass to model is called prompt
- Space available for the prompt (prompt length) is context window.
- output of a model is called completion.
- Act of using a model to generate some output is called inference.

### Transformer Architecture

- Transformers changed the Generative Ai field. 
- Before this architecture, previous architectures like rnn were not good in generating and also it is not possible to scale them to understand the context. 
- Transformer came and changed everything. It is scalable, can process paralelly, gives attention to meaning of input words it is processing.

- First we need to use tokenizer to convert text into numbers, this must be same tokenizer used while pretraining.
- Next passes to embedding layer.Each token id is matched into multi dimensional vector.
- You also add positional encoding to preserve the word order.
- Next we will pass this result into self attention layer, Here model analyzes the relation btw tokens and input sequence to get capture contexual dependencies btw words.
- But this doesnt happend only once.Transformer has multi headed self attention . This means multiple sets of attention heads are learnt in parallel independently of each other.This varies from 12-100.Each self attention head will learn different aspects of language.
- Now that attention has been applied to input data,this output is processed through feed forward network.
- Output of this layer is vector of logits proportional to probability score for each token in tokenizer dictionary(means list of full vocab words)
- We pass this logits to softmax layer where they are normalized into probability score of each word in vocabulary.
- Now from this list we will get token output based on highest probability score word. But this can be varied with some parameters like temp etc
- Now this is just basic overview of each layers. Let us see how three different variations of transformer architecture works with prompt input given.


# Generative AI Project LifeCycle:


1. Define the problem.
2. Chose the model
3. Adapt and align the model
4. Evaluate
5. Optimize and deploy model for inference
6. Build llm powered applications

### 3 Ways to Adapt and Align the model:
1. Prompting : 
Example is Few shot incontext learning
2. Fine tuning : Example is lora
3. Align with human feedback:Example is RLHF

### Prompting


- We can engineer our prompt for the model by Providing examples inside the context window is called in-context learning. 
- It can be zero , one or few shot inference, (one shot means one example input and output in prompt).
- If model doesnt behave well till may be 5 shots, then fine tuning or going for higher parameter model or better model is the option.

### Generative Configurations:


- These are the configuration parameters that we can use to influence the final decision of model about next word generation.
- These are only invoked during inference time and are not the model parameters learned during training time i.e. trainable parameters 


1. max new tokens: to limit the  number of tokens model will generate, if end of sequence token reaches before this limit , then ofcourse model wont predict further. if not this will the the limit.

- Output from softmax layer is a probability distribution of entire dictionary of words that model uses. Consider this is the dictionary 

    {mango:0.3,apple:0.2, banana:0.05, hello:0.02, bus:0.01,....}
    
    By default most llm selects top probability word as next word. i.e. greedy approach or Greedy Sampling.
    
    But this is ok for short generation but if prediction is long then susseptible to repeated words,
    
    So it is better to have some randomness or creative predictions. i.e Random Sampling
    
    But if we just do it randomly across all words, then some irrelevant word will be predicted.
    
    So we can limit the dictionary of words for prediction and can chose our random word from that. 
    
    So Top-p and Top-k are such parameters to select an output using random-weighted sampling.
   
2. Top-k : This is one way we can limit random sampling and make output sensible. So in this we specify a number, and then we limit our dictionary of words into that number , then we randomly sample from that limited dictionary.

      For example , if we have top_k=3, then out of top 3 probabilistic words i.e mango, apple and banana, one will be randomly selected as next token.

3. Top-p : Here also instead of number, we fix a probability, i.e for example like 0.5, so we select top  probability words whose sum of probability is within 0.5, and randomly sample from those. 
    That means sum of probability of mango and apple is 0.5, so only these words will fit, so one from these two will be randomly selected as next token. cumulative probability<=p
    
4. Temperature: This parameter is also used to control the randomness of model output. higher the value , higher the randomness, and vice versa.

    This parameter influence the shape of probability distribution the model calculates for predicting next      token.
    
    Temperature value is the scaling factor that is supplied in the softmax layer of the model.
    
    Default value will be 1 that means distribution is scaled to 1. if we decrease temp, <1 means, distribution will be small, that means probability will be distributed among smaller number of words for the model to chose from, from these words model will chose from random sampling.
    If temp >1, probability distribution will be broader and less peaked, so more words to chose from for the next word. 


# Fine Tuning LLM : 

Fine tuning can be done to imporve the performance of the existing model for specific use case

- One particular most common strategy used in finetuning is instruction fine tuning.
- In this we give prompt:completion pair i.e labelled dataset for the model
- We can generate this instruction labelled dataset using several prompt templates available.
- Fine tuning is a supervised process where as pretraining is a semi supervised one.
- Instruction fine tuning where all the model weights are updated is called full fine tuning.
- We split the training data into train ,test and and validation set., then after training we do evaluatation
- Training data is passed to model and we get output, as we know output of the model is probability distribution across tokens. we compare this with our label and use cross entropy function to calculate loss and to update weights using back propogation. So this will happen across multiple batches of training data and epochs as usual.


- So these fine tuning can be done on single tasks or for multiple tasks.
- If you are sure you need the model for only one task say sentiment analysis, then take around 1000 sized dataset and train it.It doesnt require much data.
- But the issue with this is it may lead to catastropic forgetting , that means fine tuned model may forget what it has learnt in pretraining and cannot perform other tasks that it was trained on. So in our model will give good results on sentiment analysis but it may not be able to perform NER for example.
- To avoid catastropic forgetting , if that is problem to you, then we can go with fine tuning on multiple tasks or also PEFT.
- In Multi task fine tuning , we provide mixed dataset {prompt:completion} for multiple tasks, so model will be good at multiple tasks , but we need lots of dataset , examples for these are generally the instruct models got from base models, they do in same process.
- Now after fine tuning, we need to evaluate the training/model as we do in typical ml , but in these outputs are not deterministic, so we cant use metrics like accuracy directly. 
- For specific tasks, there are certain evaluation methods like for example, for summarization:Rogue score,
For translation: Bleu score. 
- These methods are not suitable for all tasks, and also not best either, so check for new evaluation metrics.

- General Purpose models can be evaluated using Benchmarks like Glue,Super Glue, HELM,MMLU
- Now these are used to measure performance of llms on wide variety of things like its intelligence, math solving ability, coding ablility, world knowledge, common sense,reasoning etc. Models will be tested on these different things based on dataset in these benchmarks.
- These are the things that we see in model release page also, where they will list the score of model for these benchmarks.



### PEFT:

- Full fine tuning often required lot of hardware to store trainable weights allong with optimizer states,gradients,forward activations,temporary memory. 
- In PEFT:Parameter Efficient Fine tuning, as the name suggests this is the efficient way of fine tuning a llm, with not much of tradeoffs in efficiency compared to full finetuning.Often this can be performed on single GPU also.
- In this we only finetune a subset of existing parameters or add new layer of parameters and finetune them. That means either all or most of the weights are kept forzen based on the type of peft method we are doing.
- PEFT is less prone to catastropic forgetting since all/many parameters are left unchanged unlike full fine tuning,
- PEFT weights can be trained for each task, and therefore swapped based on requirement during inference.this leads to adoptation of model for multiple tasks.


- There are several methods that can be used for PEFT based on different tradeoffs,
    1. Memory Efficiency
    2. Parameter Efficiency
    3. Training Speed
    4. Inference Costs
    5. Model Performance.
   
    

- Three main classes of Peft methods:
    1. Selective Methods: Select subset of initial model parameters for finetune. We can identify these trainable parameters like either select a parameter type or specific layer or certain component based on requirement.
    2. Reparameterize Methods: This also works with initial model parameters but it reduces the the parameters to train by creating new low rank representation of origin network weights. Example for this is LORA.
    3. Additive MEthods: In these methods, Pretraine model weights will remain same, but we add addition trainable parameters/layers. In this there are two main approaches.
        - Adapters add new trainable params/layers to the architecure of the model inside encode/decoder components after feedforward layer/attention layer. 
        - Other method is soft prompts methods ,one example is Prompt tuning.
        This keeps architecture fixed but focus of manipulating inputs.This can be done by adding trainable parameters to prompt embeddings or by retraining embedding weights.

#### LORA:

Low Rank Adaptation of Large language model is a peft technique that falls in reparametrized category.



- Training Steps:
    1. Freeze most of the model weights
    2. Inject two rank decomposition matrices A and B, multiply them. B*A to get the same dimension of the weights they are moifying.
    3. Train the weights of smaller matrices using supervised training like how we saw before.
-  For Inference, Add these two origin weights and update them in the original matrix. updated weights+B*A
- 
We can apply lora just on self attention component and it wil be good enough based on research, but in general we can also apply on other components like feed forward layer etc.

 we can take rank r between 4 to 32, still no proper rules actually, but based on best approaches, this range will be fine since less r means less trainable params and vice versa. and also more r doesnt guarantee best performance. So this range will be good,

- During inference, we can interchange this pair of matrices we trained based on tasks. For different tasks, you can have different trained rank decomposition matrices, during inference , take that respective pair, multiple them, add and update them in the origin weights and then make inference on that particular task.

Adapters:
Let us take BERT large model: Bidirectional Encoder only Representation of Transformers
This has 24 transformer blocks.It has around 354Million parameters

input-> T1 -> T2 .. -> T24  -> output

Each Transformers block will have ulti head attention, feed forward layer , layer normalization etc.

Now we cant finetune all 345M params.
What i can do is after multi head attention and feed forward layer, i can add an Adapter in each transformer layer.

Adapter helps adapt to a finetuned task.
Adapters are small, trainable modules crafted to be both lightweight and modular, seamlessly integrating at various points within an LLM architecture.By selectively fine-tuning these specific modules rather than the entire model, adapters facilitate the customization of pre-trained models for distinct tasks, requiring only a minimal increase in parameters.
So coming to peft, we add an adapter and we update the weights of only that adapter while freezing all weights of the model.
Now instead of doing adapter weight finetunig, we can 

#### Prompt Tuning:

- Dont confuse this with prompt engineering.
- This is also a peft technique, here we dont change the architecture of the model.i.e model weights left untouched.
- As the model size increases,i.e for larger models this approach achieves as good result as full fine tuning.

- Prompt engineering requires lot of manual effort to try different prompts.
- In prompt tuning, we add addition trainable parameters/tokens called soft promts to our input prompt and leave it upto supervised learning for determining their optimal value.
- By supervised learning, model learns the value for these tokens that maximises performace for given task.
- It get preappended to our embeding vecotr that represnets input prompt and it has same lenght as embedding vector.
- Somewhere between 20-100 such tokens can be enough for good performance.
- Embedding vector of the soft prompts gets updated over time to optimize model completion
- Similar to lora, we can train different soft prompts for different tasks and use them during inference.
- During inference, we preappend input prompt with trained soft prompt i.e learned tokens , 

# RLHF:

- models  including toxic language in completion,aggressive response,providing dangerous info,offensive response.These happens bcz models are trained with wide variety of data which may contain such data.
- models are not honesh in response, even though it doesnt know, it says in confidence. sometimes it doesnt understand context and doesnt give helful answer .

- So Helpful, Honesty, Harmless are important for a model response.
- So additional method/fine tuning is required for taking care of these these things in the response so that it should be helful, honest,and harmless and give relevant response

- Popular technique to fine tune a model on human feedback is called RLHF- Reinforcement Learning from Human Feedback. 
- Instruction Tuned Model--> RLHF--> Human aligned LLM

Some advantages or use cases of RLHF:
1. Relevant or helpful response
2. Harmless response
3. Honest response
4. Personalization of response or model. For example: personalized ai assistant.

Lets see what reinforcement learning means in basic first:
- Reinforcement learning is a type of machine learning in which an agent learns to make decisions related to a specific goal by taking actions in an environment, with the objective of maximizing some notion of a cumulative reward.
- basically like an agent will be given a goal to achieve, so it learns iteratively to make better decision in this process , this will be based on reward system.
- In this framework, the agent continually learns from its experiences by taking actions, observing the resulting changes in the environment, and receiving rewards or penalties, based on the outcomes of its actions. By iterating through this process, the agent gradually refines its strategy or policy to make better decisions and increase its chances of success.

Steps in RLHF:

1. Define model alignment criteria
2. Obtain human feedback
2. Train a reward model.
3. Fine tune the llm with reinforcement learning using the trained reward model.

Summary of RLHF process:
In this process, you make use of a reward model to assess an LLMs completions of a prompt data set against some human preference metric, like helpful or not helpful. Next, you use a reinforcement learning algorithm, in this case, PPO, to update the weights off the LLM based on the reward assigned to the completions generated by the current version off the LLM. You'll carry out this cycle of a multiple iterations using many different prompts and updates off the model weights until you obtain your desired degree of alignment. Your end result is a human aligned LLM that you can use in your application

- For evaluation of rlhf, we can take model before and after rlhf fine tuning and have the pass data to reward model to evaluate the score.

# Buidling LLM based applications

From here we will see how we can build llm based applications and different topics which will come along this way.

###  Model Optimization:

- Now that we have a llm ready, we need to make it ready for inference. but the large models will have higher inference time and takes lot of memory and cant be used for edge devices like may be even mobile apps.
- We may need to optimize the llm(reduce size of llm) for faster inference , lower costs, less memory, easy deployment,suitable for edge devices.low latency and other reasons.
- So this process will be a tradeoff between accuracy and performace, since optimization will reduce performance, but how much reduction depends on the type of optimization we do.

3 main types of optimization:
- Distillation
- Quantization
- Pruning

### RAG:


- RAG-Retrieval Augemented Generation
- RAG is a framework for building llm prowered systems that make use of external datasources. Be it you want to connect your databse or some documents or connect llm to web,or connecting your proprietary data, we can do all these in RAG. 



- imagine you have an llm which has a knowdledge cutoff until some point, can we want our llm make use of web to access unknown info rather than generating some stupid things, this will be way.
- and also imagine in your company , there are lot of data, which you want to provide to the model, but cant train on it since that is again a tideous process, you can use RAG process so the model just retireves relevant data from datasources and generate content based on that data.

- Data source can be anything, but generally vector datastores are useful for llm since text data.


Example RAG pipeline:
user query->query encoder->external data source->combine relevant info with user query->llm->model response

- Take the user query
- Convert it into embedings
- pass this to a vector embedded data source which was vector representation of all our data that we want model to access.We can also chunk the data if it doesnt fit in context window before embedding. we gets symantically related info may be using consine similarity from data source.
- take the output from this source, combine with the origin user query/prompt.
- Pass this combined prompt to llm and now model will generate a response based on data within the context window.


- LLMs can also solve steps of actions one by one, imagine customer asks for password update, we can call relevant api based on this prompt, get new password from user, pass it to db, update the passord, give response to user. so all these are possible from the llm based applications.
- LLMs are generally not good with reasoning or complext tasks, so it is good if we break the task using chain of thought prompting. For example you want to solve a big math problem. give step by solution may be few  shot prompting, then ask it solve the problem, so giving example with intermediate steps will help.

- ReAct: Prompting strategy combines chain of thought prompting with action planning, basically to execute a complex task which invloes connecting with external sources, taking step by step actions, connecting with apis etc.
- In REACT, prompts are structured in such a way: question, thought, action, observation.
- question: problem that requires advanced reasoning and multiple steps to solve
- thought:reasoning step on model will tackle the problem and identify actions to take 
- action: external task that model carries out from set of allowed actions
- observation: result of carrying out the action.
- Now these steps can happen multiple times to solve a complex problem.
- Frameworks like langchain will help in these steps.
- Sometimes there can be many paths to take, so to interpret inputs from user and determine what tool to complete the task, this can be done using Agents, in langchain, agents are present for ReAct.

In [18]:
class CustomFactorial:
    def __init__(self, n):
        self.n = n

    def factorial(self, n=None):
        # Use the instance's n if no argument is provided
        if n is None:
            n = self.n
        if n == 0:
            return 0
        elif n > 0:
            if n == 1:
                return 1
            else:
                return n * self.factorial(n - 1)
        else:  # n < 0
            if n == -1:
                return -1
            else:
                return n * self.factorial(n + 1)

# Example usage:
cf = CustomFactorial(3)
print(cf.factorial())  # Output: -24


6


In [ ]:
class CustomFactorial:
    def factorial(self, n):
        # Base case for zero
        if n == 0:
            return 0
        # For positive n, standard factorial
        elif n > 0:
            if n == 1:
                return 1
            else:
                return n * self.factorial(n - 1)
        # For negative n, product from n up to -1
        else:  # n < 0
            if n == -1:
                return -1
            else:
                return n * self.factorial(n + 1)

# Example usage:
cf = CustomFactorial()
print(cf.factorial(5))    # Output: 120
print(cf.factorial(0))    # Output: 0
print(cf.factorial(-4))   # Output: -24  (i.e., -4 * -3 * -2 * -1)
